# 🛫 **Proyek Analisis Data: Air Quality**
- **Nama:** Ngakan Putu Widyasprana
- **Email:** ngakanputu39@gmail.com
- **ID Dicoding:** ngakanwp

Data yang akan digunakan adalah Air Quality yang sumbernya dapat diakses pada tautan berikut [Air Quality](https://github.com/marceloreis/HTI/tree/master).

---

## 🙋🏻‍♀️ **Pertanyaan Bisnis**



Air Quality adalah dataset mengenai kualitas udara pada 12 daerah di beijing ([sumber informasi](https://archive.ics.uci.edu/dataset/501/beijing+multi+site+air+quality+data)) yang sudah dicocokan dengan data di China Meteorological Administration, rentang waktunya 1 Maret 2013 hingga 28 Februari 2017.

**Parameter pada Dataset:**

1. Month: Bulan pada saat pengukuran data (angka 1-12).
2. Day: Hari pada saat pengukuran data (angka 1-31).
3. Hour: Jam pada saat pengukuran data (angka 0-23).
4. PM2.5: Konsentrasi partikel halus PM2.5 (μg/m³). PM2.5 adalah 5 partikel udara dengan diameter ≤ 2.5 μm yang dapat membahayakan kesehatan.
5. PM10: Konsentrasi partikel udara PM10 (μg/m³), yaitu partikel dengan diameter ≤ 10 μm.
6. SO2: Konsentrasi sulfur dioksida (μg/m³), gas yang dapat berasal dari pembakaran bahan bakar fosil.
7. NO2: Konsentrasi nitrogen dioksida (μg/m³), gas polutan hasil pembakaran kendaraan atau industri.
8. CO: Konsentrasi karbon monoksida (μg/m³), gas beracun yang dihasilkan dari pembakaran tidak sempurna.
9. O3: Konsentrasi ozon (μg/m³), gas yang dapat membentuk kabut fotokimia dan memengaruhi kualitas udara.
10. TEMP: Suhu udara pada saat pengukuran (°C).
11. PRES: Tekanan udara pada saat pengukuran (hPa).
12. DEWP: Suhu titik embun (°C), yaitu suhu di mana udara mencapai kejenuhan dan kondensasi terjadi.
13. RAIN: Presipitasi atau curah hujan yang terjadi (mm).
14. Wd: Arah angin pada saat pengukuran (kompas, seperti N, S, E, W).
15. WSPM: Kecepatan angin (m/s).
16. Station: Nama lokasi tempat pemantauan kualitas udara dilakukan.

Pertanyaan Bisnis:
- **Bagaimana tren polusi udara pada masing-masing daerah di Beijing?**
- **Kapan waktu udara terparah terjadi di masing-masing daerah di Beijing?**
- **Apakah ada korelasi antar parameter pemicu (TEMP, DEWP, RAIN, Wd, WSPM) dengan parameter polusi (PM2.5, PM10, SO2, NO2, CO, O3)? Jika ada, maka bagaimana korelasinya?**

## 📚 **Library Data Analisis**

In [ ]:
!pip install pandas numpy seaborn matplotlib plotly scipy

In [ ]:
# Library Data Retrieving and Modification
import zipfile
import os

# Library for Data Preprocessing
import pandas as pd
import numpy as np

# Library for Visualization
import scipy
from scipy.interpolate import interp1d
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

In [ ]:
def generate_requirements_txt():
    """
    This function checks the versions of specific libraries used in the project,
    generates a `requirements.txt` file with the required dependencies and their versions,
    and then displays the content of that file.

    Libraries checked:
    - pandas
    - numpy
    - seaborn
    - matplotlib
    - plotly
    - scipy

    Steps performed by the function:
    1. Print the version of each of the required libraries.
    2. Use `pip freeze` to generate a list of installed packages and filter them to include only the required libraries.
    3. Overwrite the `requirements.txt` file with the filtered libraries and their versions.
    4. Display the content of the generated `requirements.txt` file in the Colab environment.

    The `requirements.txt` file is saved in the current working directory of the Colab environment.
    """

    # Print the versions of the libraries (optional, for checking)
    print("pandas version:", pd.__version__)
    print("numpy version:", np.__version__)
    print("scipy version:", scipy.__version__)
    print("seaborn version:", sns.__version__)
    print("matplotlib version:", matplotlib.__version__)
    print("plotly version:", plotly.__version__)

    # Step 3: Generate `requirements.txt` for your specific libraries
    import subprocess

    # Get a list of installed libraries and their versions using pip freeze
    installed_packages = subprocess.check_output(["pip", "freeze"]).decode("utf-8").splitlines()

    # List of required packages
    required_packages = [
        'pandas', 'numpy', 'seaborn', 'matplotlib', 'plotly', 'scipy'
    ]

    # Filter out only the required packages from the installed ones
    filtered_packages = [
        package for package in installed_packages if any(pkg in package for pkg in required_packages)
    ]

    # Step 4: Overwrite the `requirements.txt` file with the filtered packages
    with open('requirements.txt', 'w') as f:  # 'w' ensures the file is overwritten
        f.write("\n".join(filtered_packages))

    # Step 5: Verify that `requirements.txt` is saved in the Colab environment
    !cat requirements.txt

generate_requirements_txt()

## 🔍 **Data Wrangling**

### 📩 **Gathering Data**

Sebelum memulai menjalankan kode, silahkan lakukan upload file **Air-quality-dataset.zip** untuk bisa dilakukan proses unpack.

In [ ]:
def unpack_zip(path):
    """
    Unpacks the specified zip file to a 'content' directory.

    This function checks if the given path exists. If not, it raises a
    FileNotFoundError. It then ensures that a 'content' directory is
    present, creating it if necessary, and extracts the zip file contents
    into this directory.

    Parameters:
      path (str): The path to the zip file to be unpacked.

    Raises:
      FileNotFoundError: If the specified zip file does not exist.
    """

    # Check if the path exists
    if not os.path.exists(path): raise FileNotFoundError(f"File '{path}' does not exist.")

    # Define the extraction destination
    extraction_path = "content"

    # Create the 'content' folder if it doesn't exist
    if not os.path.exists(extraction_path): os.makedirs(extraction_path)

    # Extract the zip file
    with zipfile.ZipFile(path, 'r') as zip_ref: zip_ref.extractall(extraction_path)

In [ ]:
unpack_zip('data/Air-quality-dataset.zip')

#### ⏩ **Insight Data Gathering**

---

Tidak ada insight yang bisa digali dalam sesi kali ini. Sesi kali ini berfokus untuk pengumpulan data yakni dengan cara melakukan unzip data ke folder yang sudah ditetapkan. Untuk akses data dan penilaian data dilanjutkan pada tahap berikutnya.

### 🔨 **Asessing Data**

Proses data asessing adalah proses untuk mengindentifikasi masalah pada data baik itu permasalahan data kosong, duplikat, dan data belum menggunakan format yang tepat. Pada sesi kali ini akan ada beberapa proses penilaian data, yakni sebagai berikut:
1. Missing Column
2. Duplicated Column
4. Invalid Column

In [ ]:
def check_assessing_data(df):
    """
    This function assesses a DataFrame for null values, null records, duplicate rows,
    and provides an overview of the DataFrame's structure.

    Parameters:
      df (pandas.DataFrame): The DataFrame to be assessed.

    Returns:
      None
    """

    # Check Null, Duplicated, and Info Data
    null_data = df.isnull().sum()
    null_records = df[df.isnull().any(axis=1)]
    duplicate_data = df.duplicated().sum()

    # Print Data
    print("Null Data:")
    print(null_data)
    print("\nNull Records:")
    print(null_records)
    print("\nDuplicate Data:")
    print(duplicate_data)
    print("\nCheck Column Data:")
    df.info()

#### **1. Data Air Quality (DAQ) - Aotizhongxin (AO)**

In [ ]:
# Read Data City -- Code: AO

daq_ao = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Aotizhongxin_20130301-20170228.csv', sep=',')
daq_ao.head()

In [ ]:
daq_ao.describe()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_ao)


---

Kesimpulannya Insight Data `daq_ao`:

1. Terdapat 3249 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **2. Data Air Quality (DAQ) - Changping (CH)**

In [ ]:
# Read Data City -- Code : CH

daq_ch = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Changping_20130301-20170228.csv', sep=',')
daq_ch.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_ch)


---

Kesimpulannya Insight data `daq_ch`:
1. Terdapat 2383 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **3. Data Air Quality (DAQ) - Dingling (DL)**

In [ ]:
# Read Data City -- Code: DL

daq_dl = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Dingling_20130301-20170228.csv', sep=',')
daq_dl.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_dl)


---

Kesimpulannya Insight data `daq_dl`:
1. Terdapat 3758 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **4. Data Air Quality (DAQ) - Dongsi(DG)**

In [ ]:
# Read Data City -- Code: DG

daq_dg = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Dongsi_20130301-20170228.csv', sep=',')
daq_dg.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_dg)


---

Kesimpulannya Insight data `daq_dg`:
1. Terdapat 4726 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **5. Data Air Quality (DAQ) - Guanyuan (GY)**

In [ ]:
# Read Data City -- Code: GY

daq_gy = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Guanyuan_20130301-20170228.csv', sep=',')
daq_gy.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_gy)


---

Kesimpulannya Insight data `daq_gy`:
1. Terdapat 2801 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **6. Data Air Quality (DAQ) - Gucheng (GU)**

In [ ]:
# Read Data City -- Code: GU

daq_gu = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Gucheng_20130301-20170228.csv', sep=',')
daq_gu.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_gu)


---

Kesimpulannya Insight data `daq_gu`:
1. Terdapat 2560 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **7. Data Air Quality (DAQ) - Huairou (HU)**

In [ ]:
# Read Data City -- Code: HU

daq_hu = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Huairou_20130301-20170228.csv', sep=',')
daq_hu.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_hu)


---

Kesimpulannya Insight data `daq_hu`:
1. Terdapat 3356 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **8. Data Air Quality (DAQ) - Nongzhanquan (NZ)**

In [ ]:
# Read Data City -- Code: NZ

daq_nz = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Nongzhanguan_20130301-20170228.csv', sep=',')
daq_nz.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_nz)


---

Kesimpulannya Insight data `daq_nz`:
1. Terdapat 1950 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **9. Data Air Quality (DAQ) - Shunyi (SH)**

In [ ]:
# Read Data City -- Code: SH

daq_sh = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Shunyi_20130301-20170228.csv', sep=',')
daq_sh.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_sh)


---

Kesimpulannya Insight data `daq_sh`:
1. Terdapat 4870 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **10. Data Air Quality (DAQ) - Tiantan (TI)**

In [ ]:
# Read Data City -- TI

daq_ti = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Tiantan_20130301-20170228.csv', sep=',')
daq_ti.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_ti)


---

Kesimpulannya Insight data `daq_ti`:
1. Terdapat 2221 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **11. Data Air Quality (DAQ) - Wanliu (WL)**

In [ ]:
# Read Data City -- Code: WL

daq_wl = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Wanliu_20130301-20170228.csv', sep=',')
daq_wl.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_wl)


---

Kesimpulannya Insight data `daq_wl`:
1. Terdapat 4430 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### **12. Data Air Quality (DAQ) - Wanshouxigong (WX)**

In [ ]:
# Read Data City -- Code: WX

daq_wx = pd.read_csv('content/PRSA_Data_20130301-20170228/PRSA_Data_Wanshouxigong_20130301-20170228.csv', sep=',')
daq_wx.head()

> Melakukan pengecekan apakah terdapat data kosong, jumlah record kosong, data duplikat, dan informasi data

In [ ]:
check_assessing_data(daq_wx)


---

Kesimpulannya Insight data `daq_wx`:
1. Terdapat 2296 record data yang null atau kosong.
2. Tidak terdapat duplikat.
3. Column year, month, day, dan hour bisa diubah ke tipe data datetime[ns] sehingga lebih compact dalam analisis timeseries data.

#### ⏩ **Insight Asessing Data**

---

| Data Name | Null Value | Duplicate Data | Invalid Format |  
|----------|----------|----------|----------|
| AO | 3249 | 0 | Date Invalid |
| CH | 2383 | 0 | Date Invalid |
| DL | 3758 | 0 | Date Invalid |
| DG | 4726 | 0 | Date Invalid |
| GY | 2801 | 0 | Date Invalid |
| GU | 2560 | 0 | Date Invalid |
| HU | 3356 | 0 | Date Invalid |
| NZ | 1950 | 0 | Date Invalid |
| SH | 4870 | 0 | Date Invalid |
| TI | 2221 | 0 | Date Invalid |
| WL | 4430 | 0 | Date Invalid |
| WX | 2296 | 0 | Date Invalid |

---

**Conclusion Statement -- Insight Asessing Data:**
1. Dikarenakan jumah nilai *null* atau kosong yang sangat tinggi, maka jika memilih opsi untuk menghilangkan atau menghapus data akan menjadi kehilangan informasi data yang besar. Oleh karena itu, nilai *null* akan dilakukan penghitungan dan pengisian dengan **metode interpolarisasi** untuk mengisi data kosong.

2. Untuk mengatasi format tanggal yang belum sesuai nanti akan dilakukan proses penggabungan dan penyesuaian data dengan format [YY-MM-DD
 HH:mm].

3. Dapat dilihat juga semua data baik dari parameter polusi dan parameter pemicu polusi pada masing-masing daerah terdapat yang hilang atau missing dengan jumlah yang cukup banyak. Untuk analisis lanjutan nanti seperti pengecekan data *outlier* dan data *correlation* atau korelasi antar data akan dilakukan di segmen "**Further Asessing Data and Cleaning**" di proses Data Cleaning.

### 🧹 **Data Cleaning**

Proses data cleaning kali ini akan dilakukan sesuai dengan hasil proses asessing data sebelumnya. Dalam prosesnya nanti akan dibagi menjadi 3 tahapan proses:

1. Proses mengubah kolom tahun, bulan, hari, dan jam menjadi satu kolom baru bernama datetime.
2. Proses untuk mengatasi data *null* atau kosong.
3. Proses lanjutan setelah 2 proses dijalankan yakni mengatasi data *outlier* dan pengecekan korelasi antar kolom.

Dari ketiga proses tersebut nantinya akan diambil sebagai insight dan dilanjutkan ke tahap ke-4 yakni EDA (Exploratory Data Analysis).

#### **1. Deal with Invalid Date Format - Merge and Compact**

In [ ]:
def combine_date(df):
    """
    Combine year, month, day, and hour columns in a DataFrame into a single datetime column.

    This function assumes the DataFrame contains columns named 'year', 'month', 'day', and 'hour',
    and these columns contain integer values. It combines them into a new column called 'datetime'
    in the format YYYY-MM-DD HH:00:00.

    Parameters:
    - df (pd.DataFrame): The input DataFrame with separate 'year', 'month', 'day', and 'hour' columns.

    Returns:
    - pd.DataFrame: The original DataFrame with an added 'datetime' column.
    """
    # Convert year, month, day, and hour column into new datetime column
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    return df

In [ ]:
all_daq = [
    daq_ao, daq_ch, daq_dl, daq_dg,
    daq_gy, daq_gu, daq_hu, daq_nz,
    daq_sh, daq_ti, daq_wl, daq_wx
]

all_daq = [combine_date(df) for df in all_daq]

In [ ]:
all_daq[0]['datetime']

#### **2. Deal with Null Value - Interpolation**

In [ ]:
def interpolation_linear(df):
    """
    Perform linear interpolation for numeric columns and forward/backward fill for object columns.

    This function assumes the DataFrame contains a 'datetime' column, which is set as the index.
    It also drops the 'No', 'year', 'month', 'day', and 'hour' columns if present, and performs
    the following:
    - For numeric columns (float64 or int64), missing values are interpolated using the 'linear' method.
    - For object columns, missing values are filled using forward-fill (ffill) and backward-fill (bfill).

    Parameters:
    - df (pd.DataFrame): The input DataFrame with a 'datetime' column and potential missing values.

    Returns:
    - pd.DataFrame: The DataFrame with missing values handled according to column type.
    """
    # Set datetime as Index First
    df.set_index('datetime', inplace=True)

    # Drop column that are not related anymore in DataFrame
    df.drop(['No', 'year', 'month', 'day', 'hour'], axis=1, inplace=True)

    # Do looping for column 'numeric'-- Linear or 'object'--ffill() and bfill()
    # column to use different method of interpolation
    for column in df.columns:
      if df[column].dtype in ['float64', 'int64']:

        df[column] = df[column].ffill().bfill()

        if df[column].isna().any():
            df[column] = df[column].interpolate(method='linear')

      elif df[column].dtype == 'object':
          df[column] = df[column].ffill().bfill()

    return df

In [ ]:
all_daq = [
    daq_ao, daq_ch, daq_dl, daq_dg,
    daq_gy, daq_gu, daq_hu, daq_nz,
    daq_sh, daq_ti, daq_wl, daq_wx
]

all_daq = [interpolation_linear(df) for df in all_daq]

In [ ]:
# Loop all to check the result

# Define the DataFrame Name
daq_names = [
    "daq_ao", "daq_ch", "daq_dl", "daq_dg",
    "daq_gy", "daq_gu", "daq_hu", "daq_nz",
    "daq_sh", "daq_ti", "daq_wl", "daq_wx"
]

# Make Variabel to Store Missing Value
missing_values = {}

# Loop Trough All Dataframe and Insert in missing_values
for name, df in zip(daq_names, all_daq):
    missing_values[name] = df.isna().sum()

# Convert and Transpose the missing_values
missing_values_df = pd.DataFrame(missing_values)
missing_values_df = missing_values_df.T

# Reset Index and Rename 'Index' into 'DataFrame'
missing_values_df.reset_index(inplace=True)
missing_values_df.rename(columns={"index": "DataFrame"}, inplace=True)

missing_values_df

In [ ]:
all_daq[0].head()

#### **3. Further Asessing and Cleaning Data**

Setelah berhasil dalam menangani masalah pada data yakni *Invalid Date Format* dan *Null Value* selanjutnya adalah menangani masalah lanjutan seperti data *outlier* dan mengecek korelasi antar data.

Akan tetapi sebelum itu, pastikan sekali lagi semua format data baik itu `int64`, `datetime[ns]`, `float64`, dan `object` sudah benar. Salah satu contohnya pada tipe data `float64` yang data seharusnya berupa angka desimal seperti 0.0, 800.0, 500.0 dan lainnya.

In [ ]:
def ensure_correct_dtypes(df):
    """
    Ensures all columns in a list of DataFrames have the correct data types.

    This function checks each DataFrame and performs the following operations:
    - If the index is of datetime type, it is reset and moved to a regular column.
    - Numeric columns are converted to numeric, coercing errors (i.e., invalid values become NaN).
    - Datetime columns (including columns with 'date' in their name) are converted to datetime, coercing errors.
    - Object (string) columns are converted to strings.

    Parameters:
    - dfs (list of pd.DataFrame): A list of DataFrames to be processed.

    Returns:
    - list of pd.DataFrame: A list of DataFrames with corrected data types.
    """

    # Define variabel for dataframe that already corrected
    corrected_dataframes = []

    # Looping data in list of dataframe and correct all data into
    # corresponding data type column.
    for data in df:
        data = data.copy()

        # Reset index datetime to regular column
        if pd.api.types.is_datetime64_any_dtype(data.index):
            data.reset_index(inplace=True)

        # Looping all column in dataframe
        for col in data.columns:

            # Check and correct data in column numeric
            if pd.api.types.is_numeric_dtype(data[col]):
                data[col] = pd.to_numeric(data[col], errors='coerce')

            # Check and correct data in column datetime
            elif pd.api.types.is_datetime64_any_dtype(data[col]) or 'date' in col.lower():
                data[col] = pd.to_datetime(data[col], errors='coerce')

            # Check and correct data in column object
            elif pd.api.types.is_object_dtype(data[col]):
                data[col] = data[col].astype(str)

        # Store all corrected column into variabel corrected_dataframes
        corrected_dataframes.append(data)

    return corrected_dataframes

In [ ]:
corrected_df = ensure_correct_dtypes(all_daq)

In [ ]:
corrected_df[0].info()

In [ ]:
corrected_df[0]

##### 🧶 **Check and Deal With the Outlier Data**

In [ ]:
def detect_outliers_iqr(df):
    """
    Detects outliers in a DataFrame using the Interquartile Range (IQR) method.

    This function identifies outliers based on the IQR, which is the
    range between the 25th and 75th percentiles (Q1 and Q3) of the data.
    Any data points that are below the lower bound (Q1 - 1.5 * IQR) or above the upper
    bound (Q3 + 1.5 * IQR) are considered outliers.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numerical data for outlier detection.

    Returns:
    - outliers_df (pd.DataFrame): A DataFrame containing the rows with outliers.
    - non_outliers_df (pd.DataFrame): A DataFrame containing the rows without outliers.
    """

    # Define outliers variabel and select just numeric column
    # in DataFrame
    outliers = pd.DataFrame(index=df.index)
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Loop through the all numeric column and calculate
    # IQR (Interquartile) Method
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Select data that related to lower bound of data and
        # upper bound of data
        outliers[col] = (df[col] < lower_bound) | (df[col] > upper_bound)

    # Select data with outliers act as filter to seperated
    # outliers and non-outliers data
    outliers_df = df[outliers.any(axis=1)]
    non_outliers_df = df[~outliers.any(axis=1)]

    return outliers_df, non_outliers_df

In [ ]:
def plot_boxplots(df):
    """
    Plots boxplots for all numeric columns in the given DataFrame.

    This function generates a boxplot for each numeric column in the DataFrame,
    displaying the distribution, median, and potential outliers. It uses `matplotlib`
    to create the plots and adds titles and labels for clarity.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numeric data.

    Returns:
    - None: The function displays boxplots but does not return any values.
    """

    # Select data that just numeric column in dataframe
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Check the numeric column is not empty or 0
    if len(numeric_cols) == 0:
        print("No numeric columns to plot.")
        return

    # Looping through numeric column and do box-plot visualization
    for col in numeric_cols:
        plt.figure(figsize=(10, 6))
        plt.boxplot(df[col].dropna(), vert=False, patch_artist=True, boxprops=dict(facecolor='lightblue'))
        plt.title(f'Box Plot for {col}', fontsize=16)
        plt.xlabel(col, fontsize=14)
        plt.grid(axis='x', linestyle='--', alpha=0.7)

    plt.show()


In [ ]:
def winsorize_outliers(df, lower_percentile=5, upper_percentile=95):
    """
    Winsorizes outliers in the numeric columns of a DataFrame.

    This function replaces the outliers in each numeric column with values
    at the specified lower and upper percentiles. Outliers are defined as
    values that are below the lower percentile or above the upper percentile.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numeric data.
    - lower_percentile (int, optional): The lower percentile threshold (default is 5).
    - upper_percentile (int, optional): The upper percentile threshold (default is 95).

    Returns:
    - pd.DataFrame: A DataFrame with winsorized values for outliers.
    """

    # Get numeric column from dataframe
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Calculate lower bound and upper bound
    # in every numeric columns in dataframe
    lower_bound = df[numeric_cols].quantile(lower_percentile / 100)
    upper_bound = df[numeric_cols].quantile(upper_percentile / 100)

    # Make dataframe to store result of winsorized
    df_winsorized = df.copy()

    # Looping through the numeric columns and do winsorized data in dataframe
    for col in numeric_cols:
        df_winsorized[col] = df_winsorized[col].clip(lower=lower_bound[col], upper=upper_bound[col])

    return df_winsorized

In [ ]:
# Set index for dataframe after winsorized
# will be choose index 0 until 11
index_winsorized=0

###### **Check Outlier Data - 1 DataFrame Example**

In [ ]:
plot_boxplots(corrected_df[index_winsorized])

In [ ]:
corrected_df[index_winsorized].describe()

In [ ]:
outliers, non_outliers = detect_outliers_iqr(corrected_df[index_winsorized])
outliers.describe()

In [ ]:
non_outliers.describe()

In [ ]:
winsorized_df = winsorize_outliers(corrected_df[index_winsorized], lower_percentile=10, upper_percentile=90)
winsorized_df.describe(include='all')

In [ ]:
plot_boxplots(winsorized_df)

###### **Deal Outliers Data - All DataFrame**

In [ ]:
all_winsorized_df = [winsorize_outliers(df, lower_percentile=10, upper_percentile=90) for df in corrected_df]

##### 🧶 **Check The Correlation**

In [ ]:
def check_correlation(df):
    """
    Plots a correlation matrix heatmap for the numeric columns of a DataFrame.

    This function calculates the Pearson correlation coefficient between each
    pair of numeric columns, excluding specified non-numeric columns, and
    visualizes the correlation matrix as a heatmap. Correlation coefficients
    range from -1 to 1, with values closer to 1 or -1 indicating stronger correlations.
    - Positive values indicate a positive correlation.
    - Negative values indicate a negative correlation.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numeric and non-numeric data.

    Returns:
    - None: This function only displays the heatmap and does not return any values.
    """

    # Drop column that not will use in make correlation matrix
    df_cor = df.drop(columns=['station', 'wd', 'datetime'])

    # Get correlation value and store in correlation variabel
    correlation_matrix = df_cor.corr()

    # Visualize the correlation value into correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Correlation Matrix Heatmap')
    plt.show()

In [ ]:
# Choose and set index to choose dataframe
# that will visualize from 0-11
index_correlation = 0

###### **Check Correlation Data - 1 DataFrame Example**

In [ ]:
check_correlation(all_winsorized_df[index_correlation])

#### ⏩ **Insight Data Cleaning**

---

Setelah melakukan proses data cleaning dimulai dari proses mengubah format date menjadi lebih *compact*, mengatasi nilai *null* atau kosong, melakukan analisis lanjutan mengenai data *outlier* dan korelasi data didapatkan beberapa hal menarik sebagai berikut:

1. Pengubahan format tanggal membuat kolom menjadi lebih ringkas dan membantu mempermudah dalam analisis lanjutan, seperti digunakan saat proses menyelesaikan *null* value dengan membuat kolom datetime menjadi index dari datanya.

2. Menyelesaikan masalah *null* value sudah dapat mengatasi kekosongan data pada data yang bertipe numerik dan object. Untuk tipe numerik menggunakan metode interpolasi linear sedangkan untuk data tipe object menggunakan metode interpolasi *forward fill* digabungkan dengan *backward fill*.

3. Setelah data lengkap, analisis lanjutan yang bisa dilakukan adalah mengatasi data *outlier* dan pengecekan korelasi antar datanya. Untuk data outlier diatasi dengan metode winsoring untuk menggantikan nilai data berdasarkan dengan batas bawah dan batas atas percentil-nya. Dengan ini maka data *outlier* sudah dapat ditangai dengan baik, ini dapat dilihat dari visualisasi box-plot yang tidak menunjukan indikasi adanya data *outlier*.

4. Proses untuk pengecekan korelasi antara kolom juga dapat dilihat pada hasil *correlation matrix*. Pada matrik terdapat beberapa kolom, baik kolom parameter polusi dan kolom parameter pemicu polusi berkaitan satu sama lain walau kecil, untuk analisis lebih lanjut akan pada proses EDA nantinya.  

## ⌚ **EDA (Exploratory Data Analysis)**

EDA atau Exploratory Data Analysis adalah proses untuk menggali insight dari data yang digunakan untuk menjawab pertanyaan bisnis. Dalam proses EDA terdapat beberapa proses yakni tahap persiapan dan proses eksplorasi data.

### **1. Tahap Persiapan EDA**

In [ ]:
selected_dfs = all_winsorized_df[:12]
final_df = pd.concat(selected_dfs, ignore_index=True)
final_df

In [ ]:
p_polution_final_df = final_df.drop(columns=['TEMP','PRES','DEWP','RAIN','wd','WSPM'])
pc_polution_final_df = final_df.drop(columns=['PM2.5','PM10','SO2','NO2','CO','O3'])

In [ ]:
p_polution_final_df.head()

In [ ]:
pc_polution_final_df.head()

In [ ]:
final_df.to_csv("dashboard/final_df.csv", index=False)

### **2. Tahap Eksplorasi Data**

#### 🥨 **Parameter Polusi**

Parameter polusi terdiri dari beberapa kolom yakni PM2.5, PM10, SO2, NO2, CO, dan O3. Keenam kolom ini menandakan bahwa pada suatu daerah memiliki kondisi udara yang baik atau tidak.

In [ ]:
p_polution_final_df.describe()

In [ ]:
p_polution_final_df.head()

In [ ]:
p_polution_final_df.groupby(by=['station']).agg({
    'PM2.5': 'mean',
    'PM10': 'mean',
    'SO2': 'mean',
    'NO2': 'mean',
    'CO': 'mean',
    'O3': 'mean',
}).reset_index()

Dapat dilihat pada tabel diatas, terdapat total 12 kota di Beijing dengan 6 indikator kualitas dari udara nya. Secara sekilas, untuk persebaran nilai indikatornya cukup merata pada satu daerah dengan daerah yang lain, oleh karena itu, coba untuk mencari nilai maksimal dan minimal tiap indikator pada masing-masing daerah.

In [ ]:
for col in p_polution_final_df.columns:
    if col not in ['station', 'datetime']:
      group_data = p_polution_final_df.groupby(by=['station'])[col].mean()
      max_station = group_data.idxmax()
      max_value = group_data.max()
      print(f"Kota Dengan Nilai {col} Tertinggi : {max_station}, Nilai: {max_value}")

      min_station = group_data.idxmin()
      min_value = group_data.min()
      print(f"Kota Dengan Nilai {col} Terendah : {min_station}, Nilai: {min_value}\n")

Dari 6 indikator, yang memegang posisi 3 indikator tertinggi adalah daerah Dongsi dengan indikator PM2.5, SO2, dan CO tertinggi. Sedangkan daerah yang memegang indikator terrendah terbanyak adalah Dingling, dengan 4 indikatornya adalah PM2.5, PM10, NO2, dan CO.


In [ ]:
daily_df = p_polution_final_df.copy()
daily_df.set_index('datetime', inplace=True)
daily_mean = daily_df.groupby('station').resample('D').mean().reset_index()
daily_mean.head()

In [ ]:
weekly_df = p_polution_final_df.copy()
weekly_df.set_index('datetime', inplace=True)
weekly_mean = weekly_df.groupby('station').resample('W').mean().reset_index()
weekly_mean.head()

In [ ]:
montly_df = p_polution_final_df.copy()
montly_df.set_index('datetime', inplace=True)
montly_mean = montly_df.groupby('station').resample('ME').mean().reset_index()
montly_mean.head()

In [ ]:
annual_df = p_polution_final_df.copy()
annual_df.set_index('datetime', inplace=True)
annual_mean = annual_df.groupby('station').resample('YE').mean().reset_index()
annual_mean.head()

Setelah mengidentifikasi secara kasar mengenai daerah dengan indikator tertinggi dan terendah, dalam data parameter polusi juga dapat diubah menjadi rata-rata data dalam satuan jam, hari, minggu, bulan dan tahun. Ini memberikan ruang analisis menjadi lebih beragam pada skala yang lebih kecil.

#### 🥨 **Parameter Pemicu Polusi**

Parameter pemicu polusi adalah beberapa parameter pendukung yang dapat memicu sebuah polusi seperti temperatur (TEMP), tekanan udara(PRES), titik embun (DEWP), tingkat curah hujan (RAIN), arah angin (wd), dan kecepatan angin (WSP).

In [ ]:
pc_polution_final_df.describe(include='all')

In [ ]:
pc_polution_final_df.head(10)

In [ ]:
pc_polution_final_df.groupby(by='wd').agg({
    'station' : 'nunique',
    'TEMP': 'mean',
    'PRES': 'mean',
    'DEWP': 'mean',
    'RAIN': 'mean',
    'WSPM': 'mean',
}).reset_index()

Dapat dilihat pada tabel, untuk arah mata angin terdapat pada semua daerah di beijing dengan total ada 15 arah mata angin. Nilai pada indikator rain menunjukkan nilai 0 untuk semua data sehingga informasi yang bisa digali tidak ada, Ini juga sama dengan indikator station yang menunjukkan kesamaan data pada semua indikator.

Selain itu, pada tabel sebelumya, arah mata angin berubah-ubah di tiap jamnya.

In [ ]:
for col in pc_polution_final_df.columns:
    if col not in ['station', 'datetime', 'wd', 'RAIN']:
      group_data = pc_polution_final_df.groupby(by=['wd'])[col].mean()
      max_station = group_data.idxmax()
      max_value = group_data.max()
      print(f"Arah Mata Angin Dengan Nilai {col} Tertinggi : {max_station}, Nilai: {max_value}")

      min_station = group_data.idxmin()
      min_value = group_data.min()
      print(f"Kota Dengan Nilai {col} Terendah : {min_station}, Nilai: {min_value}\n")

Dapat dilihat pada hasil perulangan diatas, didapatkan bahwa arah mata angin NW mendapatkan nilai terendah pada indikator TEMP dan DEWP, lalu indikator tertinggi pada WSPM. Lalu ada juga arah mata angin SSE yang menjadi terendah di PRES dan tertinggi di DEWP.

#### 🥨 **Korelasi Antar Parameter**

In [ ]:
final_df.drop(columns=['RAIN'], inplace=True)
check_correlation(final_df)

Secara keseluruhan, parameter polusi saling berkaita kuat antara satu dengan yang lainnya, sedangkan parameter pemicu polusi tidak demikian. Beberapa hubungan antar parameter seperti indikator PM2.5 dan PM10 berhubungan sangat lemah pada indikator titik embun (DEWP), lalu indikator SO2, NO2, dan CO berhubungan lemah juga dengan indikator tekanan angin (PRES). Namum indikator O3 berhubungan cukup kuat pada 2 indikator suhu (TEMP) dan kecepatan angin (WSPM).

Jika melihat pada bagian parameter pemicu polusi, hanya indikator temperatur (TEMP) berhubungan kuat dengan indikator lainnya yakni indikator titik embun (DEWP).

#### 🥨 **Geospatial Analysis**

In [ ]:
def validate_inputs(year, month, day, hour):
    """
    Validates the user input filters for year, month, day, and hour.

    This function checks that the necessary filters are specified for each level
    of time filtering. The user must provide filters in a logical sequence:
    - Year is required if Month or Day are specified.
    - Month is required if Day is specified.
    - Day is required if Hour is specified.

    Parameters:
    - year (int or None): The selected year for filtering.
    - month (int or None): The selected month for filtering.
    - day (int or None): The selected day for filtering.
    - hour (int or None): The selected hour for filtering.

    Raises:
    - ValueError: If the required filters are not specified in a valid order.
    """

    # Ensure that day is specified only if month and year are provided
    if day is not None and (month is None or year is None):
        raise ValueError("To filter by day, specify both year and month.")

    # Ensure that month is specified only if year is provided
    if month is not None and year is None:
        raise ValueError("To filter by month, specify the year.")

    # Ensure that hour is specified only if day, month, and year are provided
    if hour is not None and (day is None or month is None or year is None):
        raise ValueError("To filter by hour, specify year, month, and day.")

In [ ]:
# Step 1: Define Stations Coordinates
# Station coordinates get from outsources like google maps and google earth.
# There is still have some issue that coordinat maybe wrong.
stations_coords = {
    "Aotizhongxin": (41.731242, 123.456778),
    "Changping": (40.221, 116.2312),
    "Dingling": (40.28998423518348, 116.2393424781757),
    "Dongsi": (40.10208908941478, 116.31657335910373),
    "Guanyuan": (39.94113871141321, 116.3610710753842),
    "Gucheng": (39.91270053243136, 116.1868698799306),
    "Huairou": (43.06043347888646, 117.46726428196578),
    "Nongzhanguan": (39.93978579546827, 116.46859787734736),
    "Shunyi": (40.151287025024715, 116.69280368021326),
    "Tiantan": (39.88189413732897, 116.42047003643812),
    "Wanliu": (39.99843210685499, 116.25774299569612),
    "Wanshouxigong": (39.90816416629832, 116.26439549963654)
}

# Step 2: Make Visualization that According to Spesific Date Location
# This step purpose to make filter based on year, month, day, and hour with
# level of filter is Year -> Month -> Day -> Hour.
selected_year = 2016
selected_month = 11
selected_day = None
selected_hour = None

# Step 3: Define a Function that Validate the Users Input
# This step crucial to make rank filter to make spesific location
# visualization later.
validate_inputs(selected_year, selected_month, selected_day, selected_hour)

# Step 4: Prepare DataFrame with Coordinates
spatial_df = final_df.copy()
spatial_df["latitude"] = spatial_df["station"].map(lambda x: stations_coords[x][0])
spatial_df["longitude"] = spatial_df["station"].map(lambda x: stations_coords[x][1])
spatial_df["datetime"] = pd.to_datetime(spatial_df["datetime"])

# Step 5: Filter the Data According the User Input
# At this section, users input are the variable that not None.
# The filter work is based on rank filter first by year until hour.
filtered_df = spatial_df.copy()
if selected_year is not None:
    filtered_df = filtered_df[filtered_df["datetime"].dt.year == selected_year]
if selected_month is not None:
    filtered_df = filtered_df[filtered_df["datetime"].dt.month == selected_month]
if selected_day is not None:
    filtered_df = filtered_df[filtered_df["datetime"].dt.day == selected_day]
if selected_hour is not None:
    filtered_df = filtered_df[filtered_df["datetime"].dt.hour == selected_hour]

# Step 6: Determine Grouping and Hover Columns for Visualization
# This step is crucial to make a new dataframe that spesific for
# visualization later, so the data is small column that include
# station name, latitude, longitude, and rank filter selection.
group_cols = ["station", "latitude", "longitude"]
if selected_hour is not None:
    filtered_df.loc[:, "hour"] = filtered_df["datetime"].dt.hour
    group_cols = ["hour"] + group_cols
    hover_cols = ["hour", "PM2.5"]
elif selected_day is not None:
    filtered_df.loc[:, "day"] = filtered_df["datetime"].dt.day
    group_cols = ["day"] + group_cols
    hover_cols = ["day", "PM2.5"]
elif selected_month is not None:
    filtered_df.loc[:, "month"] = filtered_df["datetime"].dt.month
    group_cols = ["month"] + group_cols
    hover_cols = ["month", "PM2.5"]
elif selected_year is not None:
    filtered_df.loc[:, "year"] = filtered_df["datetime"].dt.year
    group_cols = ["year"] + group_cols
    hover_cols = ["year", "PM2.5"]

# Step 7: Group and Aggregate Variabel PM2.5
# This step will use example of 1 variabel for fast and less computation
# visualization. The groupping dataframe will accordding to level rank
# that input by users.
filtered_df = (
    filtered_df.groupby(group_cols)
    .agg({"PM2.5": "mean"})
    .reset_index()
)

# Step 8: Plot the Visualization on the Map Using Ploty
fig = px.scatter_mapbox(
    filtered_df,
    lat="latitude",
    lon="longitude",
    color="PM2.5",
    size="PM2.5",
    hover_name="station",
    hover_data=hover_cols,
    mapbox_style="carto-positron",
    title="Time-Series Geospatial Data",
    zoom=5,
)

fig.show()

Dapat dilihat pada gambar peta diatas, ada beberapa titik lokasi yang memiliki perbedaan warna titik pada satu daerah dengan daerah yang lainnya. Perbedaan warna ini dipengaruhi oleh tingkat konsentrasi parameter polusi PM2.5 nya pada suatu daerah. Semakin pekat maka semakin tinggi, sedangkan sebaliknya semakin cerah/terang maka semakin rendah.

### ⏩ **Insight EDA (Eksploratory Data Analysis)**

Secara keseluruhan dalam proses ekploratory dibagi menjadi 2 tahap yakni tahapan persiapan dan eksplorasi. Untuk tahap persiapan dilakukan persiapan data dengan menggabungkan semua dataframe daerah menjadi satu untuk memudahkan proses analisis. Lalu, membagi kembali menjadi 2 dataframe dengan 2 kategori berbeda yakni : Data Parameter Polusi dan Data Parameter Pemicu Polusi.

- Pada hasil eksplorasi data parameter polusi didapatkan banyak hal terkait dengan daerah Dongsi yang menjadi pemilik indikator tertinggi yakni pada PM2.5, SO2, dan CO, sedangkan daerah Dingling dengan paling banyak indikator polusi terendah yakni PM2.5, PM10, NO2, dan CO.

- Pada eksplorasi parameter polusi juga didapatkan bahwa data dapat diubah dengan ukuran dalam satuan jam, hari, minggu, bulan, dan tahun. Dengan ini menandakan bisa membuat visualisasi yang mendetail dan rinci dalam implementasi nanti.

- Dilanjutkan dengan proses eksplorasi parameter pemicu polusi, didapatkan bahwa indikator station dan curah hujan (RAIN) tidak memiliki perlakuan yang berbeda untuk data jadi dapat dihilangkan.

- Selain itu didapatkan juga arah mata angin (wd) yang memiliki nilai tertinggi dan terendah terdapat pada arah mata angin SSE dan NW.

- Selanjutnya adalah melihat korelasi data setelah digabungkan dengan menghilangkan indikator curah hujan (RAIN) yang tidak berdampak signifikan. Didapatkan bahwa parameter polusi saling berhubungan antara satu sama lain, sedangkan parameter pemicu polusi yang berhubungan dengan baik hanya indikator temperatur (TEMP) dengan indikator titik embun (DEWP).

- Yang terakhir adalah analisis tambahan pada analisis geospasial menggunakan ploty. Pada analisis ini bertujuan untuk melihat perbandingan nilai variabel PM2.5 pada masing-masing daerah jika dipetakan dalam ruang spasial. Untuk variabel bisa digunakan yang lain seperti PM10, dan lainnya agar visualisasi menjadi lebih mendetail.



## 💬 **Data Visualization dan Explanatory**

Pada tahapan Data Visualization dan Explanatory ini akan menjelaskan insight yang ditemukan pada data untuk menjawab pertanyaan bisnis yang sudah diajukan sebelumnya, untuk pertanyaan bisnisnya sebagai berikut:

1. Bagaimana tren polusi udara pada masing-masing daerah di Beijing?
2. Kapan waktu udara terparah terjadi di masing-masing daerah di Beijing?
3. Apakah ada korelasi antar parameter pemicu (TEMP, DEWP, RAIN, Wd, WSPM) dengan parameter polusi (PM2.5, PM10, SO2, NO2, CO, O3)? Jika ada, maka bagaimana korelasinya?

Ketiga pertanyaan ini akan dijawab dalam 3 segmen berbeda dengan segmen terakhir adalah kesimpulan akhir dari Data Visualization dan Explanatory.

Untuk dataframe yang digunakan adalah `final_df`, `p_polution_final_df`, dan `pc_polution_final_df`.

### **Pertanyaan 1: Bagaimana tren polusi udara pada masing-masing daerah di Beijing?**

In [ ]:
def aggregate_data(df, visualization_type, component_date={'year': None, 'month': None, 'days': None}):
    """
    Aggregates the given DataFrame based on the specified visualization type (Annual, Monthly, Daily, or Hourly).

    This function computes aggregated statistics (mean) on air quality data, grouped by time-based components
    such as year, month, day, or hour. The function can be customized to aggregate data for a specific year,
    month, or day if necessary.

    Parameters:
    - df : pandas.DataFrame
        The input DataFrame containing air quality data. Must include a 'datetime' column and pollutant data columns
        (e.g., 'PM2.5', 'PM10').

    - visualization_type : str
        The type of aggregation to perform. Accepted values are:
        - 'Annual' : Aggregates data by year.
        - 'Monthly' : Aggregates data by month, optionally for a specific year.
        - 'Daily' : Aggregates data by day, optionally for a specific month and year.
        - 'Hours' : Aggregates data by hour, optionally for a specific day, month, and year.

    - component_date : dict, optional (default is {'year': None, 'month': None, 'days': None})
        A dictionary specifying the specific year, month, and day for monthly, daily, and hourly aggregations.
        The dictionary may include:
        - 'year' : The specific year for monthly, daily, or hourly aggregation.
        - 'month' : The specific month for daily or hourly aggregation.
        - 'days' : The specific day for hourly aggregation.
        If not provided, the function will aggregate based on all available data.

    Returns:
    - agg_df : pandas.DataFrame
        The aggregated DataFrame based on the chosen visualization type. The data is grouped by relevant time units
        (e.g., year, month, day, or hour), and the mean of each pollutant is computed for each group.

    - id_vars : list
        A list of columns used as identifier variables for the aggregation. These columns depend on the selected
        aggregation type (e.g., ['station', 'year'] for annual aggregation).

    - labels : list
        A list containing the name of the time unit used in the aggregation, along with its descriptive label
        (e.g., ['year', 'Year']).

    Raises:
    - ValueError : If 'datetime' column is missing in the DataFrame.
    - ValueError : If an invalid 'visualization_type' is provided.
    - ValueError : If required values for year, month, or day are missing when performing monthly, daily, or hourly aggregation.
    """

    # Raise Value Error if datetime Column not present.
    if 'datetime' not in df.columns:
      raise ValueError("'datetime' column is missing in the DataFrame.")

    # Detect Year, Month, Day, and Hour
    df.loc[:, 'year'] = df['datetime'].dt.year
    df.loc[:, 'month'] = df['datetime'].dt.month
    df.loc[:, 'day'] = df['datetime'].dt.day
    df.loc[:, 'hour'] = df['datetime'].dt.hour

    # Detect Unique Component Datetime
    unique_years = df['year'].nunique(),
    unique_months = df['month'].nunique(),
    unique_days = df['day'].nunique(),
    unique_hours = df['hour'].nunique()

    # Initialize variables
    agg_df = None
    id_vars = []
    labels = []

    # Use match-case for aggregation
    match visualization_type:
        case 'Annual':
            agg_df = df.groupby(['station', 'year']).mean().reset_index()
            id_vars = ['station', 'year']
            labels = ['year', 'Year']

        # Case for Single Year and More than One Year
        case 'Monthly' if df['year'].nunique() == 1:
            agg_df = df.groupby(['station', 'month']).mean().reset_index()
            id_vars = ['station', 'month']
            labels = ['month', 'Month']
        case 'Monthly':
            if component_date['year'] is None:
                raise ValueError(f"Year is required for monthly aggregation.")
            selected_year = component_date['year']
            agg_df = df[df['year'] == selected_year].groupby(['station', 'month']).mean().reset_index()
            id_vars = ['station', 'month']
            labels = ['month', 'Month']

        # Case for Single Month and More than One Months
        case 'Daily' if df['month'].nunique() == 1:
            agg_df = df.groupby(['station', 'day']).mean().reset_index()
            id_vars = ['station', 'day']
            labels = ['day', 'Day']
        case 'Daily':
            if component_date['year'] is None or component_date['month'] is None:
                raise ValueError(f"Year and Month are required for daily aggregation.")
            selected_year = component_date['year']
            selected_month = component_date['month']
            agg_df = df[(df['year'] == selected_year) & (df['month'] == selected_month)].groupby(['station', 'day']).mean().reset_index()
            id_vars = ['station', 'day']
            labels = ['day', 'Day']

        # Case for Single Day and More than One Days
        case 'Hours' if df['day'].nunique() == 1:
            agg_df = df.groupby(['station', 'hour']).mean().reset_index()
            id_vars = ['station', 'hour']
            labels = ['hour', 'Hour']
        case 'Hours':
            if component_date['year'] is None or component_date['month'] is None or component_date['days'] is None:
                raise ValueError(f"Year, Month, and Day are required for hourly aggregation.")
            selected_year = component_date['year']
            selected_month = component_date['month']
            selected_day = component_date['days']
            agg_df = df[(df['year'] == selected_year) & (df['month'] == selected_month) & (df['day'] == selected_day)].groupby(['station', 'hour']).mean().reset_index()
            id_vars = ['station', 'hour']
            labels = ['hour', 'Hour']

        # Case for if all case above not match
        case _:
            raise ValueError("Invalid visualization type. Choose from 'Annual', 'Monthly', 'Daily', 'Hours'.")

    return agg_df, id_vars, labels


def ploty_line_visualization(data, id_vars, labels, parameters, title_template="Average Concentration Data by Station and {}"):
    """
    Creates a Plotly line chart to visualize air quality data by station and a specified time unit (e.g., year, month, hour).

    Parameters:
    - data : pandas.DataFrame
        The input data containing station names, time variables, and pollutant concentrations.
        The DataFrame must include columns corresponding to the time unit(s), pollutant concentrations, and station identifiers.

    - id_vars : list
        The identifier variables used to group the data (e.g., ['station', 'year']).

    - labels : list
        A list containing the name of the time unit column and its descriptive label (e.g., ['year', 'Year']).

    - parameters : list
        A list of pollutant columns to include in the visualization (e.g., ['PM2.5', 'PM10']).

    - title_template : str, optional (default: "Concentration Data by Station and {}")
        A string template for the chart title, with the placeholder `{}` being replaced by the descriptive label of the time unit.

    Returns:
    - plotly.graph_objects.Figure
        A Plotly figure object representing the line chart, visualizing the pollutant concentrations by station over the specified time unit(s).
    """

    # For Better Visualization will Using Long Format with Function melt()
    df_melted = data.melt(
        id_vars=id_vars,
        value_vars=parameters,
        var_name='Pollutant',
        value_name='Concentration'
    )

    # Define Free Variabel to Set the Visualization Plot
    if 'month' in df_melted.columns:
        month_names = {
            1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
            7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
        }
        df_melted['month_name'] = df_melted['month'].map(month_names)
        labels[0] = 'month_name'
        tick_mode = 'array'
        tick_val = list(month_names.values())
        tick_text = tick_val
        tick_angle = 0
    elif 'hour' in df_melted.columns:
        df_melted['AM_PM'] = df_melted['hour'].apply(lambda x: f"{x % 12 or 12} {'AM' if x < 12 else 'PM'}")
        labels[0] = 'AM_PM'
        tick_mode = 'array'
        tick_val = df_melted['AM_PM'].unique()
        tick_text = tick_val
        tick_angle = 90
    else:
        tick_mode = 'linear'
        tick_val = None
        tick_text = None
        tick_angle = None

    fig = px.line(
        df_melted,
        x=f"{labels[0]}",
        y='Concentration',
        color='station',
        labels={labels[0]: labels[1], 'Concentration': 'Concentration', 'station': 'Station'},
        line_shape='linear'
    )

    # Set the Layout Visualization
    fig.update_layout(
        title=title_template.format(labels[1]),
        legend_title='Station',
        template='simple_white',
        width=900,
        height=450,
        title_font=dict(size=16),
        xaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=False,
            gridwidth=1,
            gridcolor='white',
            tickmode=tick_mode,
            tickformat=".0f",
            tickvals=tick_val,
            ticktext=tick_text,
            tickangle=tick_angle
        ),
        yaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgrey'
        ),
        legend=dict(
            title='Station',
            font=dict(size=12),
        ),
        hovermode='closest',
        plot_bgcolor='white',
        paper_bgcolor='ghostwhite'
    )

    return fig

Pertanyaan mengenai tren polusi udara, maka dapat dijawab dengan menggunakan plot visualisasi garis dengan membandingkan antar daerah tergantung pada parameter polusinya.

In [ ]:
p_polution_final_df.head()

In [ ]:
# Step 1 : Make Filter Data
start_date = '2013-03-01 00:00:00'
end_date = '2017-03-31 23:00:00'

filter_df = p_polution_final_df.loc[(p_polution_final_df['datetime'] >= start_date) & (p_polution_final_df['datetime'] <= end_date)].copy()

In [ ]:
# Step 2: Define Variabel and Make Aggregate Data in Years, Month, Day, and Hours.
choose_visualization = 'Annual'
choose_parameter = ['PM2.5']
agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization)

# Step 3: Do Visualization
agg_data.drop(columns=['month', 'day', 'hour'], inplace=True)
fig = ploty_line_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Monthly in A Year

choose_visualization = 'Monthly'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date= {'year': 2013})
agg_data.drop(columns=['year', 'day', 'hour'], inplace=True)

fig = ploty_line_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Daily in a Month

choose_visualization = 'Daily'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date={'year': 2013, 'month': 3})
agg_data.drop(columns=['month', 'year', 'hour'], inplace=True)

fig = ploty_line_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Hourly in a Day

choose_visualization = 'Hours'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date={'year': 2013, 'month': 3, 'days': 1})
agg_data.drop(columns=['month', 'day', 'year'], inplace=True)

fig = ploty_line_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

#### 🔆 **Explanatory Pertanyaan Pertama**

Dalam segmen pertanyaan pertama ini mengenai tren pada data Air Quality - Parameter Polusi dilakukan dengan analisis berdasarkan pembagian waktunya. Berikut adalah pembagian analisisnya:
1. Pembagian Berdasarkan Tahun Keseluruhan
2. Pembagian Berdasarkan Bulan per Tahunnya
3. Pembagian Berdasarkan Tanggal per Bulannya
4. Pembagian Berdasarkan Jam per Harinya

---

##### **1. Pembagian Berdasarkan Tahun Keseluruhan**
Dapat dilihat pada visualisasi data mengenai rata-rata dalam [hitungan tahun keseluruhan](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=neQoU6oU2dxJ&line=1&uniqifier=1). Terdapat peningkatan nilai rata-rata parameter polusi yakni PM2.5 di seluruh derah beijing. Dapat dilihat juga pada grafik, 3 daerah tertinggi nilai parameter PM2.5 di tahun 2013 adalah Wanliu, Dongsi, dan Nongzhanguan. Lalu di 2017 terdapat perubahan yakni yang tertinggi adalah daerah Wanshouxigong, Dongsi, dan Nongzhanguan.  

Lalu, selain daerah dengan tingkat PM2.5 tertinggi adapun daerah yang menjadi 3 terendah yakni daerah Dingling, Huairio, dan Changping. Terutama daerah Dingling menjadi daerah satu-satunya yang konsisten menjadi yang terendah untuk tingkat indikator PM2.5.

Untuk analisis lebih lanjut bisa mencoba melihat parameter polusi lainnya seperti PM10, SO2, NO2, CO, dan O3.

##### **2. Pembagian Berdasarkan Bulan per Tahunnya**

Dapat dilihat pada visualisasi data mengenai rata-rata parameter polusi berdasarkan dari [hitungan bulan tiap tahun](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=wkb2qCIFRy8v&line=1&uniqifier=1). Pada tahun 2013, Wanliu termasuk daerah dengan rata-rata tertinggi untuk indeks PM2.5-nya, terutama puncaknya ada pada bulan Juni sebesar 109. Selain itu untuk daerah yang memiliki rata-rata tahunan terendah untuk indikator PM2.5 adalah Dingling, sempat mengalami kenaikan di bulan Juni akan tetapi untuk bulan-bulan selanjutnya konsisten menjadi terendah. Jika menilik ke tahun terbaru yakni tahun 2016 menjadi tahun dengan indeks PM2.5 terendah dikeseluruhan data, hal ini dapat dilihat pada data visualisasi bulan tiap tahunnya, terjadi penurunan drastis untuk semua daerah di bulan january ke bulan february, penurunan terjadi terhadap semua daerah.

##### **3. Pembagian Berdasarkan Tanggal per Bulannya**
Dapat dilihat pada visualisasi data mengenai rata-rata parameter polusi berdasarkan dari [hitungan hari tiap bulan](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=iKvLfZLQSGmQ&line=6&uniqifier=1). Pada tahun 2016, di bulan January ada penurunan drastis indeks PM2.5 selama 2 hari, yakni dari tanggal 1-4 January. Tentu daerah yang menjadi titik terendah adalah Dingling. Hal yang menarik juga, jika melihat data bulan february, dalam 4 hari yakni tanggal 13-16 February semua daerah konsisten rendah (linear) tidak ada peningkatan drastis atau lonjakan konsentrasi parameter PM2.5.

##### **4. PPembagian Berdasarkan Jam per Harinya**
Dapat dilihat pada visualisasi data mengenai rata-rata parameter polusi berdasarkan dari [hitungan jam perharinya](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=QBjqf4-sSgG7&line=6&uniqifier=1). Jika kita coba menilik pada data terendah di 2016, yakni dibulan Februari tanggal 13-16. Ditanggal 13, dimulai dari pukul 12AM-7AM, kecuali daerah Huario, daerah lainnya tidak mengalami lonjakan yang signifikan, lalu dari pukul 8AM-11PM, terjadi lonjakan konsentrasi paramater PM2.5, akan tetapi daerah Dingling tetap stabil tidak ada lonjakan yang tinggi. Di hari selanjutnya, semua daerah terjadi lonjakan yang signifikan untuk konsentrasi parameter PM2.5 nya, trend lonjakan paling tinggi adalah daerah Guncheng dengan titik tertingginya pukul 8 PM di 157. Di hari berikutnya kembali, pada pukul 1AM-7AM terjadi trend penurunan konsentrasi PM2.5, lalu pukul 12PM-11PM terjadi trend peningkatan konsentrasi parameter PM2.5. Lalu di hari terakhir yakni tanggal 16 dimulai pukul 12AM-6AM terjadi trend penurunan konsentrasi PM2.5 sedangkan pukul 7AM-11PM terjadi trend kenaikan konsentrasi PM2.5

### **Pertanyaan 2: Kapan waktu udara terparah terjadi di masing-masing daerah di Beijing?**

In [ ]:
def ploty_bar_visualization(data, id_vars, labels, parameters, title_template="Average Concentration Data by Station and {}"):
    """
    Visualizes pollutant concentration data in a stacked bar plot, with dynamic hover information
    based on the type of time-related data available (month, year, day, or AM/PM).

    Parameters:
    - data : pandas.DataFrame
        The input dataframe containing the data to be visualized.

    - id_vars : list of str
        The columns in the dataframe that will be used for grouping the x-axis (e.g., station name).

    - labels : list of str
        A list where the first item is the column used for coloring the bars (e.g., 'year')
        and the second item is the label for that column in the plot.

    - parameters : list of str
        A list containing the names of the pollutant parameters (e.g., ['PM2.5']) to be plotted.

    - title_template : str, optional, default="Average Concentration Data by Station and {}"
        The template for the plot's title. The `{}` placeholder will be replaced with the second
        item in the `labels` list.

    Returns:
    - plotly.graph_objects.Figure
        The Plotly figure object containing the bar plot visualization.
    """

    # Customize Month Column and Hover
    if 'month' in data.columns:
      month_names = {
          1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
          7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
      }
      data['Month'] = data['month'].map(month_names)
      hover_data={'Month': True, 'month':False}
      custom_value = 'Month'
      hover_temp = "Month: %{customdata[0]}<br>"

    # Customize Year Column and Hover
    elif 'year' in data.columns:
      hover_data=None
      custom_value = 'year'
      hover_temp= "Year: %{customdata[0]}<br>"

    # Customize Day Column and Hover
    elif 'day' in data.columns:
      hover_data=None
      custom_value = 'day'
      hover_temp= "Day: %{customdata[0]}<br>"

    # Customize Clocl Column and Hover
    else:
      data['AM_PM'] = data['hour'].apply(lambda x: f"{x % 12 or 12} {'AM' if x < 12 else 'PM'}")
      hover_data=None
      custom_value = 'AM_PM'
      hover_temp= "Hour: %{customdata[0]}<br>"

    # Do Sorting Data by Parameter
    data = data.sort_values(by=parameters[0])

    # Wrap All Variabel Visualization
    fig = px.bar(
        data,
        x=id_vars[0],
        y=parameters,
        color=labels[0],
        barmode='stack',
        hover_data=hover_data,
    )

    # Customize Hover
    fig.update_traces(customdata=data[[custom_value]].values)
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>"
        + "Concentration: %{y:.2f}<br>"
        + hover_temp
    )

    # Set the Layout Visualization
    fig.update_layout(
        title=title_template.format(labels[1]),
        legend_title='Station',
        template='simple_white',
        width=900,
        height=450,
        title_font=dict(size=16),
        xaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=False,
            gridwidth=1,
            gridcolor='white',
            tickformat=".0f",
            tickangle=45
        ),
        yaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgrey'
        ),
        legend=dict(
            title='Station',
            font=dict(size=12),
        ),
        hovermode='closest',
        plot_bgcolor='white',
        paper_bgcolor='ghostwhite'
    )

    return fig

In [ ]:
# Step 1 : Make filter data
start_date = '2013-03-01 00:00:00'
end_date = '2017-03-31 23:00:00'

filter_df = p_polution_final_df.loc[(p_polution_final_df['datetime'] >= start_date) & (p_polution_final_df['datetime'] <= end_date)].copy()

In [ ]:
# Step 2: Define Variabel and Make Aggregate Data in Years, Month, Day, and Hours.
# Aggregate in Years Total
choose_visualization = 'Annual'
choose_parameter = ['PM2.5']
agg_data, id_vars , labels = aggregate_data(filter_df, choose_visualization)

# Step 3: Do Visualization
agg_data.drop(columns=['month', 'day', 'hour'], inplace=True)
fig = ploty_bar_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Monthly in A Year
choose_visualization = 'Monthly'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date= {'year': 2013})
agg_data.drop(columns=['year', 'day', 'hour'], inplace=True)

fig = ploty_bar_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Daily in a Month
choose_visualization = 'Daily'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date={'year': 2013, 'month': 3})
agg_data.drop(columns=['month', 'year', 'hour'], inplace=True)

fig = ploty_bar_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

In [ ]:
# Aggregate and Visualize Hourly in a Day
choose_visualization = 'Hours'
choose_parameter = ['PM2.5']

agg_data, id_vars, labels = aggregate_data(filter_df, choose_visualization, component_date={'year': 2013, 'month': 3, 'days': 1})
agg_data.drop(columns=['month', 'day', 'year'], inplace=True)

fig = ploty_bar_visualization(agg_data, id_vars, labels, choose_parameter)
fig.show()

#### 🔆 **Explanatory Pertanyaan Kedua**

Dapat dilihat pada visualisasi dengan bentuk stack-bar diatas, visualisasi dilakukan dengan membagi dalam beberapa sub-bagian, pembagiannya sebagai berikut:
1. Pembagian Data dalam Hitungan Tahun.
2. Pembagian Data dalam Hitungan Bulan per Tahunnya.
3. Pembagian Data dalam Hitungan Hari per Bulannya.
4. Pembagian Data dalam Hitungan Jam per Harinya.

> *Untuk membuat penjelasan lebih khusus maka akan digunakan tanggal 1 Maret 2013 sebagai contoh pembahasan.*

---

##### **1. Pembagian Data dalam Hitungan Tahun**
Dalam visualisasi pembagian data dalam [hitungan tahun](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=L0oecyKwCX8m&line=1&uniqifier=1). Dapat dilihat pada masing-masing visualisasi yang dikelompokkan berdasarkan station memiliki nilai tertinggi parameter PM2.5 yang berbeda-beda, Salah satu contohnya pada daerah Wanliu tahun 2014 merupakan nilai tertinggi indeks polusi PM2.5-nya, sedangkan pada daerah Dongsi dan Wanshouxgong berada di tahun 2017. Untuk Indeks terendah menariknya semua daerah berada di tahun 2016. Lalu untuk indeks tertinggi yang paling banyak terjadi pada tahun 2014.    

##### **2. Pembagian Data dalam Hitungan Bulan per Tahunnya**
Dalam visualisasi pembagian data dalam [hitungan bulan per tahunnya](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=5HqAn7A_hD8v&line=1&uniqifier=1). Dapat dilihat pada masing-masing pembagian data dikelompokkan berdasarkan station dan tahunnya, yakni pada 2013. Pada tahun ini, bulan Juli merupakan bulan dengan index tertinggi pada daerah Beijing yang paling banyak, lalu disusul oleh maret dan mei. Untuk indeks terendah pada masing-masing kota bervariasi, akan tetapi khusus daerah Huairou satu-satunya memiliki indeks tertinggi di bulan Mei.

##### **3. Pembagian Data dalam Hitungan Hari per Bulannya**
Dalam visualisasi pembagian data dalam [hitungan hari per bulannya](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=qd8tj5q95k95&line=8&uniqifier=1). Dapat dilihat pada masing-masing visualisasi pembagian data dikelompokkan berdasarkan station, tahun, dan bulannya di Tahun 2013. Pada bulan Maret ini jika coba dizoom untuk melihat nilai terkecilnya maka hari pertama (Day-1)adalah hari dengan nilai indeks terkecil dari semua daerah, dilanjutkan dengan indeks tertinggi dipegang oleh hari ke-7 dan ke-17, daerah Wanliu satu-satunya daerah dengan indeks PM2.5 tertinggi di hari ke-6.

##### **4. Pembagian Data dalam Hitungan Jam per Harinya**
Dalam visualisasi pembagian data dalam [hitungan jam per harinya](https://colab.research.google.com/drive/1rCfERccLm7CMNkF_BvSk7RpwDR8Q-hCn#scrollTo=MPLsJzGh7J0T&line=1&uniqifier=1). Dapat dilihat pada masing-masing visualisasi pembagian data dikelompokkan berdasarkan station, tahun, bulan, dan harinya di tahun 2023. Ini mendapatkan data indikator PM2.5 dalam kurun waktu 24 jam di hari bersangkutan, pada contoh adalah Day ke-1. Indeks tertinggi dominan berada pada pukul 11 PM, lalu indeks terkecil pada masing-masing daerah beragam. Daerah yang cukup menarik adalah Dingling sebagai daerah dengan total indeks hariannya paling kecil yakni kurang dari 200, sedangkan daerah Guanyuan bisa sampai > 250.

### **Pertanyaan 3: Apakah ada korelasi antar parameter pemicu (TEMP, DEWP, RAIN, Wd, WSPM) dengan parameter polusi (PM2.5, PM10, SO2, NO2, CO, O3)? Jika ada, maka bagaimana korelasinya?**

In [ ]:
check_correlation(final_df)

#### 🔆 **Explanatory Pertanyaan Ketiga**

Dalam melihat keterhubungan antara variabel dapat menggunakan matrix korelasi seperti visualisasi diatas, untuk memahami matrixnya dapat dibagi menjadi 2 pemahaman:
1. Pemahaman korelasi variabel polusi dengan variabel pemicu polusi.
2. Pemahaman korelasi antar variabel polusi atau pemicu polusi.

Dengan 2 pemahaman ini akan dapat mengetahui mengenai keterhubungan parameter-parameter baik dalam lingkup sejenis atau antar lingkup. Berikut adalah pembahasan detailnya.

---

##### **1. Pemahaman Korelasi Variabel Polusi dengan Variabel Pemicu Polusi (Antar Parameter)**
Dalam memahami korelasi antar parameter kita fokus pada indikator polusi yakni (PM2.5, PM10, SO2, NO2, CO dan O3) dan parameter pemicu polusi yakni (TEMP, PRES, DEWP, WSPM). Untuk yang pertama adalah indikator PM2.5, PM2.5 dapat dilihat pada matrix, tidak memiliki hubungan atau korelasi yang baik dengan indikator pemicu polusi hampir semua nilainya negatif atau kurang dari 1, ada satu parameter yang berkorelasi positif yakni DEWP akan tetapi korelasinya terlalu kecil sebesar 0.15. Ini juga terjadi pada PM10 yang hanya berkorelasi lemah dengan DEWP. Untuk SO2, NO2, dan CO berkorelasi lemah dengan PRES. Sedangkan O3 satu-satunya variabel yang menunjukkan korelasi dengan 2 parameter pemicu lumayan tinggi mendekati 1 pada TEMP dan berkorelasi lemah pada WSPM.

##### **2. Pemahaman Korelasi Varibel Polusi atau Pemicu Polusi (Intra Parameter)**
Dalam memahami korelasi intra-parameter kita fokus pada hubungan parameter satu dengan yang lainnya dalam satu kelompok yang sama. Hal yang menonjol disini adalah, nilai korelasi parameter O3 tidak menunjukkan hubungan/korelasi yang positif antaran sesama kelompok parameternya yakni PM2.5, PM10, SO2, NO2, dan CO. Korelasi parameter O3 bernilai negatif ini dapat dikatakan bahwa O3 tidak berkorelasi sama sekali dengan parameter polusi.

Lalu jika dilihat pada parameter pemicu polusi, indikator TEMP berkorelasi kuat hanya pada indikator DEWP, sedangkan berkorelasi sangat lemah pada indikator WSPM. Indikator PRES bahkan pada matrix tidak memiliki korelasi yang kuat antar yang lainnya, hanya korelasi sangat lemah pada indikator WSPM. Untuk indikator DEWP, seperti yang dijelaskan, hanya berkorelasi tinggi pada indikator TEMP. Lalu terakhir, indikator WSPM hanya berkorelasi sangat lemah pada TEMP dan PRES.

## 🚥 **Dashboard Data Visualization - Streamlit**

### **1. Tahap Persiapan Data dan Streamlit untuk Visualisasi**

Tahap persiapan ini akan memastikan alat-alat atau tool yang digunakan sudah baik seperti text editor dan lainnya. Selain itu pastikan sudah menjalankan kode pada analisis sebelumnya sehingga data sudah siap untuk dilakukan pemodelan dashboard.

Untuk itu hal-hal yang disiapkan terlebih dahulu:

1. DataFrame `final_df` yang menampung keseluruhan tabel sudah diubah menjadi .csv untuk memudahkan pengembanan dashboard. Untuk dataframe ini bisa dilakukan upload terlebih dahulu atay menjalankan kode keseluruhan analisis sebelumnya.

2. Jika menggunakan platform cloud seperti colab maka perlu konfigurasi pihak ketiga seperti **ngrok** agar dapat diakses dalam proses pembuatannya, jika secara local maka cukup streamlit saja.

3. Streamlit hanya berjalan pada ekstensi file .py bukan .ipynb jadi untuk dapat menggunakannya pada platform jupiter notebook bisa mengubah hasil kode menjadi .py terlebih dahulu, dengan `%%writefile nama_file.py`.

4. Khusus untuk penggunaan ngrok sebagai pihak ketiga, diperlukan pembuatan akun dahulu untuk mendapatkan token authentifikasinya, lalu melakukan konfigurasi token `!ngrok authtoken kode_token`.

5. Sebagai tambahan, jika sudah memiliki `final_df.csv`, `aqi-logo`, `app.py`, maka lakukan upload pada google colab dan silahkan langsung menuju ke Tahap 3 - Menjalankan Dasboard segmen *Start Server*.

6. Tidak lupa untuk menjalankan kode pada *Segmen Library Data Analisis* sebelum memulai menjalankan Streamlit di goole colab.

In [ ]:
!pip install -q streamlit pyngrok

In [ ]:
# Authentification
!ngrok authtoken kode_token

In [ ]:
final_df = pd.read_csv('dashboard/final_df.csv')
final_df.head()

### **2. Tahap Implementasi Streamlit**

In [ ]:
%%writefile dashboard/dashboard.py

# Library Data Retrieving and Modification
import zipfile
import os

# Library for Data Preprocessing
import pandas as pd
import numpy as np

# Library for Visualization
from scipy.interpolate import interp1d
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Library for Dashboard
import streamlit as st

# """
# Segement 1 : Add Function for Visualization

# This section will add a function that will use to processing the dataframe,
# make visualization, and others.
# """

def load_dataframe(filepath):
    """
    Load and preprocess a DataFrame from a CSV file.

    Parameters:
    - file_path : str
        The file path to the CSV file containing the DataFrame.

    Returns:
    - pandas.DataFrame
        A DataFrame sorted by 'datetime', with datetime columns converted.
    """

    # Read the dataframe
    all_df = pd.read_csv(filepath)

    # Ensure the selected column is datetime format
    datetime_columns = ["datetime"]

    for column in datetime_columns:
      all_df[column] = pd.to_datetime(all_df[column])

    return all_df

def filter_data_level(df, level):
    """
    Filters the given DataFrame based on the specified levels (year, month, day).

    This function dynamically displays Streamlit selectboxes for each filter level
    (year, month, day) based on the provided `level` list, and filters the DataFrame
    accordingly. The function assumes the DataFrame has columns `year`, `month`, and
    `day` for filtering.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the data to be filtered.
    - level (list): A list containing the filtering levels. It can include 'year',
      'month', and 'day'. The filtering occurs based on the order of the levels in the list.

    Returns:
    - pd.DataFrame: The filtered DataFrame based on the selected levels.
    """

    if 'year' in level:
        selected_year = st.selectbox("Select Year:", df['year'].unique())
        df = df[df['year'] == selected_year]
    if 'month' in level:
        selected_month = st.selectbox("Select Month:", df['month'].unique())
        df = df[df['month'] == selected_month]
    if 'day' in level:
        selected_day = st.selectbox("Select Day:", df['day'].unique())
        df = df[df['day'] == selected_day]

    return df

def filter_selected_datetime(df, all_cols):
    """
    Function to dynamically filter the DataFrame by year, month, day, and hour.

    Parameters:
    - spatial_df (pd.DataFrame): The dataframe containing datetime and spatial information.

    Returns:
    - selected_year (int or None): The selected year.
    - selected_month (int or None): The selected month.
    - selected_day (int or None): The selected day.
    - selected_hour (int or None): The selected hour.
    - filtered_df (pd.DataFrame): The filtered dataframe based on the selections.
    """

    # Initialize empty selections
    selected_year = None
    selected_month = None
    selected_day = None
    selected_hour = None

    # Select Year
    selected_year = all_cols[0].selectbox("Select Year:", options=[None] + sorted(df['datetime'].dt.year.unique()))

    # Select Month
    if selected_year is not None:
        available_months = sorted(df[df["datetime"].dt.year == selected_year]["datetime"].dt.month.unique())
    else:
        available_months = []
    selected_month = all_cols[1].selectbox("Select Month:", options=[None] + available_months)

    # Select Day
    if selected_year is not None and selected_month is not None:
        available_days = sorted(df[(df["datetime"].dt.year == selected_year) & (df["datetime"].dt.month == selected_month)]["datetime"].dt.day.unique())
    else:
        available_days=[]
    selected_day = all_cols[2].selectbox("Select Day:", options=[None] + available_days)

    # Select Hour
    if selected_year is not None and selected_month is not None and selected_day is not None:
      available_hours = sorted(df[(df["datetime"].dt.year == selected_year) & (df["datetime"].dt.month == selected_month) & (df["datetime"].dt.day == selected_day)]["datetime"].dt.hour.unique())
    else:
      available_hours=[]
    selected_hour = all_cols[3].selectbox("Select Hour:", options=[None] + available_hours)

    # Apply the selected filters to the dataframe
    filtered_df = df.copy()

    # Filter Selection
    if selected_year is not None:
        filtered_df = filtered_df[filtered_df["datetime"].dt.year == selected_year]
    if selected_month is not None:
        filtered_df = filtered_df[filtered_df["datetime"].dt.month == selected_month]
    if selected_day is not None:
        filtered_df = filtered_df[filtered_df["datetime"].dt.day == selected_day]
    if selected_hour is not None:
        filtered_df = filtered_df[filtered_df["datetime"].dt.hour == selected_hour]

    all_filter = [selected_year, selected_month, selected_day, selected_hour]

    return all_filter, filtered_df

def aggregate_data(df, visualization_type):
    """
    Aggregates the given DataFrame based on the specified visualization type (Annual, Monthly, Daily, or Hourly).

    This function computes aggregated statistics (mean) on air quality data, grouped by time-based components
    such as year, month, day, or hour. The function can be customized to aggregate data for a specific year,
    month, or day if necessary.

    Parameters:
    - df : pandas.DataFrame
        The input DataFrame containing air quality data. Must include a 'datetime' column and pollutant data columns
        (e.g., 'PM2.5', 'PM10').

    - visualization_type : str
        The type of aggregation to perform. Accepted values are:
        - 'Annual' : Aggregates data by year.
        - 'Monthly' : Aggregates data by month, optionally for a specific year.
        - 'Daily' : Aggregates data by day, optionally for a specific month and year.
        - 'Hours' : Aggregates data by hour, optionally for a specific day, month, and year.

    - component_date : dict, optional (default is {'year': None, 'month': None, 'days': None})
        A dictionary specifying the specific year, month, and day for monthly, daily, and hourly aggregations.
        The dictionary may include:
        - 'year' : The specific year for monthly, daily, or hourly aggregation.
        - 'month' : The specific month for daily or hourly aggregation.
        - 'days' : The specific day for hourly aggregation.
        If not provided, the function will aggregate based on all available data.

    Returns:
    - agg_df : pandas.DataFrame
        The aggregated DataFrame based on the chosen visualization type. The data is grouped by relevant time units
        (e.g., year, month, day, or hour), and the mean of each pollutant is computed for each group.

    - id_vars : list
        A list of columns used as identifier variables for the aggregation. These columns depend on the selected
        aggregation type (e.g., ['station', 'year'] for annual aggregation).

    - labels : list
        A list containing the name of the time unit used in the aggregation, along with its descriptive label
        (e.g., ['year', 'Year']).

    Raises:
    - ValueError : If 'datetime' column is missing in the DataFrame.
    - ValueError : If an invalid 'visualization_type' is provided.
    - ValueError : If required values for year, month, or day are missing when performing monthly, daily, or hourly aggregation.
    """

    # Raise Value Error if datetime Column not present.
    if 'datetime' not in df.columns:
      raise ValueError("'datetime' column is missing in the DataFrame.")

    # Detect Year, Month, Day, and Hour
    df.loc[:, 'year'] = df['datetime'].dt.year
    df.loc[:, 'month'] = df['datetime'].dt.month
    df.loc[:, 'day'] = df['datetime'].dt.day
    df.loc[:, 'hour'] = df['datetime'].dt.hour

    # Initialize variables
    agg_df = None
    id_vars = []
    labels = []

    # Use match-case for aggregation
    match visualization_type:
      case 'Annual':
          agg_df = df.groupby(['station', 'year']).mean().reset_index()
          id_vars = ['station', 'year']
          labels = ['year', 'Year']
          agg_df.drop(columns=['month', 'day', 'hour'], inplace=True)

      case 'Monthly':
          if df['year'].nunique() > 1:
              df = filter_data_level(df, ['year'])
          agg_df = df.groupby(['station', 'month']).mean().reset_index()
          id_vars = ['station', 'month']
          labels = ['month', 'Month']
          agg_df.drop(columns=['year', 'day', 'hour'], inplace=True)

      case 'Daily':
          if df['month'].nunique() > 1:
              df = filter_data_level(df, ['year', 'month'])
          agg_df = df.groupby(['station', 'day']).mean().reset_index()
          id_vars = ['station', 'day']
          labels = ['day', 'Day']
          agg_df.drop(columns=['month', 'year', 'hour'], inplace=True)

      case 'Hours':
          if df['day'].nunique() > 1:
              df = filter_data_level(df, ['year', 'month', 'day'])
          agg_df = df.groupby(['station', 'hour']).mean().reset_index()
          id_vars = ['station', 'hour']
          labels = ['hour', 'Hour']
          agg_df.drop(columns=['month', 'day', 'year'], inplace=True)

      case _:
          raise ValueError("Invalid visualization type. Choose from 'Annual', 'Monthly', 'Daily', 'Hours'.")

    return agg_df, id_vars, labels

def ploty_line_visualization(data, id_vars, labels, parameters, title_template="Average Concentration Data by Station"):
    """
    Creates a Plotly line chart to visualize air quality data by station and a specified time unit (e.g., year, month, hour).

    Parameters:
    - data : pandas.DataFrame
        The input data containing station names, time variables, and pollutant concentrations.
        The DataFrame must include columns corresponding to the time unit(s), pollutant concentrations, and station identifiers.

    - id_vars : list
        The identifier variables used to group the data (e.g., ['station', 'year']).

    - labels : list
        A list containing the name of the time unit column and its descriptive label (e.g., ['year', 'Year']).

    - parameters : list
        A list of pollutant columns to include in the visualization (e.g., ['PM2.5', 'PM10']).

    - title_template : str, optional (default: "Concentration Data by Station and {}")
        A string template for the chart title, with the placeholder `{}` being replaced by the descriptive label of the time unit.

    Returns:
    - plotly.graph_objects.Figure
        A Plotly figure object representing the line chart, visualizing the pollutant concentrations by station over the specified time unit(s).
    """

    # Check if the dataframe empty or not
    if data.empty:
      raise ValueError("The data provided is empty. Please check the aggregation process.")

    # For Better Visualization will Using Long Format with Function melt()
    df_melted = data.melt(
        id_vars=id_vars,
        value_vars=parameters,
        var_name='Pollutant',
        value_name='Concentration'
    )

    # Define Free Variabel to Set the Visualization Plot
    if 'month' in df_melted.columns:
        month_names = {
            1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
            7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
        }
        df_melted['month_name'] = df_melted['month'].map(month_names)
        labels[0] = 'month_name'
        tick_mode = 'array'
        tick_val = list(month_names.values())
        tick_text = tick_val
        tick_angle = 45
    elif 'hour' in df_melted.columns:
        df_melted['AM_PM'] = df_melted['hour'].apply(lambda x: f"{x % 12 or 12} {'AM' if x < 12 else 'PM'}")
        labels[0] = 'AM_PM'
        tick_mode = 'array'
        tick_val = df_melted['AM_PM'].unique()
        tick_text = tick_val
        tick_angle = 90
    else:
        tick_mode = 'linear'
        tick_val = None
        tick_text = None
        tick_angle = None

    fig = px.line(
        df_melted,
        x=f"{labels[0]}",
        y='Concentration',
        color='station',
        labels={labels[0]: labels[1], 'Concentration': 'Concentration', 'station': 'Station'},
        line_shape='linear'
    )

    # Set the Layout Visualization
    fig.update_layout(
        title=title_template,
        legend_title='Station',
        template='simple_white',
        width=900,
        height=450,
        title_font=dict(size=16),
        xaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=False,
            tickmode=tick_mode,
            tickformat=".0f",
            tickvals=tick_val,
            ticktext=tick_text,
            tickangle=tick_angle
        ),
        yaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=True,
            gridcolor='lightgrey'
        ),
        legend=dict(
            groupclick='toggleitem',
            title='Station',
            font=dict(size=12),
        ),
        hovermode='closest',
        plot_bgcolor='white',
        paper_bgcolor='ghostwhite'
    )

    return fig

def ploty_bar_visualization(data, id_vars, labels, parameters, title_template="Average Concentration Data by Station"):
    """
    Visualizes pollutant concentration data in a stacked bar plot, with dynamic hover information
    based on the type of time-related data available (month, year, day, or AM/PM).

    Parameters:
    - data : pandas.DataFrame
        The input dataframe containing the data to be visualized.

    - id_vars : list of str
        The columns in the dataframe that will be used for grouping the x-axis (e.g., station name).

    - labels : list of str
        A list where the first item is the column used for coloring the bars (e.g., 'year')
        and the second item is the label for that column in the plot.

    - parameters : list of str
        A list containing the names of the pollutant parameters (e.g., ['PM2.5']) to be plotted.

    - title_template : str, optional, default="Average Concentration Data by Station and {}"
        The template for the plot's title. The `{}` placeholder will be replaced with the second
        item in the `labels` list.

    Returns:
    - plotly.graph_objects.Figure
        The Plotly figure object containing the bar plot visualization.
    """

    # Default hover customiation
    custom_value = 'datetime'
    hover_temp = "Date: %{customdata[0]}<br>"

    # Customize Month Column and Hover
    if 'month' in data.columns:
      month_names = {
          1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
          7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
      }
      data['Month'] = data['month'].map(month_names)
      custom_value = 'Month'
      hover_temp = "Month: %{customdata[0]}<br>"

    # Customize Year Column and Hover
    elif 'year' in data.columns:
      custom_value = 'year'
      hover_temp= "Year: %{customdata[0]}<br>"

    # Customize Day Column and Hover
    elif 'day' in data.columns:
      custom_value = 'day'
      hover_temp= "Day: %{customdata[0]}<br>"

    # Customize Clocl Column and Hover
    else:
      data['AM_PM'] = data['hour'].apply(lambda x: f"{x % 12 or 12} {'AM' if x < 12 else 'PM'}")
      custom_value = 'AM_PM'
      hover_temp= "Hour: %{customdata[0]}<br>"

    # Do Sorting Data by Parameter
    data = data.sort_values(by=parameters[0])

    # Wrap All Variabel Visualization
    fig = px.bar(
        data,
        x=id_vars[0],
        y=parameters,
        color=id_vars[1],
        barmode='stack',
        hover_data=None,
    )

    # Customize Hover
    fig.update_traces(customdata=data[[custom_value]].values)
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>"
        + "Concentration: %{y:.2f}<br>"
        + hover_temp
    )

    # Set the Layout Visualization
    fig.update_layout(
        title=title_template,
        legend_title='Station',
        template='simple_white',
        width=900,
        height=450,
        title_font=dict(size=16),
        xaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=False,
            tickformat=".0f",
        ),
        yaxis=dict(
            title="",
            title_font=dict(size=14),
            tickfont=dict(size=12),
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgrey'
        ),
        legend=dict(
            title='Station',
            font=dict(size=12),
        ),
        hovermode='closest',
        plot_bgcolor='white',
        paper_bgcolor='ghostwhite'
    )

    return fig

def ploty_geospatial_visualization(filtered_df, selected_filters, parameter):
    """
    Generates a geospatial visualization (scatter plot on a map) based on the given DataFrame
    and selected filters. The plot displays pollutant data based on the selected year, month,
    day, and hour, with additional data about the stations.

    The function filters the DataFrame based on the selected filters (year, month, day, hour),
    groups the data by the relevant columns, and visualizes it using Plotly's scatter_mapbox
    function.

    Parameters:
    - filtered_df (pd.DataFrame): The input DataFrame containing the data to be visualized.
    - selected_filters (list): A list of selected filters for the visualization in the order
      [selected_year, selected_month, selected_day, selected_hour].
    - parameter (list): A list containing the column name of the pollutant (e.g., ["PM2.5"]).

    Returns:
    - fig (plotly.graph_objs.Figure): The Plotly figure object containing the geospatial
      visualization.
    """

    # Extract selected filters
    selected_year, selected_month, selected_day, selected_hour = selected_filters

    # Define Group Column and Hover for Visualization
    group_cols = ["station", "latitude", "longitude"]
    hover_cols = [parameter[0]]

    if selected_hour is not None:
        filtered_df["hour"] = filtered_df["datetime"].dt.hour
        group_cols = ["hour"] + group_cols
        hover_cols = ["hour", parameter[0]]
    elif selected_day is not None:
        filtered_df["day"] = filtered_df["datetime"].dt.day
        group_cols = ["day"] + group_cols
        hover_cols = ["day", parameter[0]]
    elif selected_month is not None:
        filtered_df["month"] = filtered_df["datetime"].dt.month
        group_cols = ["month"] + group_cols
        hover_cols = ["month", parameter[0]]
    elif selected_year is not None:
        filtered_df["year"] = filtered_df["datetime"].dt.year
        group_cols = ["year"] + group_cols
        hover_cols = ["year", parameter[0]]

    # Handle missing columns or no filters applied
    if len(group_cols) == 3:
        hover_cols = [parameter[0]]

    # Sort the Filter Dataframe
    filtered_df = (
        filtered_df.groupby(group_cols)
        .agg({parameter[0]: "mean"})
        .reset_index()
    )

    # Do Visualization
    fig = px.scatter_mapbox(
        filtered_df,
        lat="latitude",
        lon="longitude",
        color=parameter[0],
        size=parameter[0],
        hover_name="station",
        hover_data=hover_cols,
        mapbox_style="carto-positron",
        title="Geospatial Visualization Polutan Data",
        zoom=5,
    )

    # Adjust layout to remove gaps and specify the figure height
    fig.update_layout(
        height=400,
        margin={"r": 100, "t": 50, "l": 0, "b": 0},  # Adjust margin
    )

    return fig

def check_correlation(df):
    """
    Plots a correlation matrix heatmap for the numeric columns of a DataFrame.

    This function calculates the Pearson correlation coefficient between each
    pair of numeric columns, excluding specified non-numeric columns, and
    visualizes the correlation matrix as a heatmap. Correlation coefficients
    range from -1 to 1, with values closer to 1 or -1 indicating stronger correlations.
    - Positive values indicate a positive correlation.
    - Negative values indicate a negative correlation.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing numeric and non-numeric data.

    Returns:
    - None: This function only displays the heatmap and does not return any values.
    """

    # Drop column that not will use in make correlation matrix
    df_cor = df.drop(columns=['station', 'wd', 'datetime', 'RAIN'])

    # Get correlation value and store in correlation variabel
    correlation_matrix = df_cor.corr()

    # Visualize the correlation value into correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Correlation Matrix Heatmap')
    st.pyplot(plt)



# """
# Segment 2 : Make Filter Layout in Side Bar

# This section will make filter layout in side bar using streamlit method siderbar.
# for the input is start date and end date. That will pass to new dataframe main_df
# that contain data from selected date
# """

# Load DataFrame and Define Variabel for Sidebar
final_df = load_dataframe('dashboard/final_df.csv')
p_polution_final_df = final_df.drop(columns=['TEMP','PRES','DEWP','RAIN','wd','WSPM'])

# Make Filter Layout in Side Bar
with st.sidebar:

  # Set image logo
  st.image("data/aqi-logo.png")

  # Set start_date and end_date from input
  p_polution_final_df["date"] = pd.to_datetime(p_polution_final_df["datetime"]).dt.date
  min_date = final_df["datetime"].min()
  max_date = final_df["datetime"].max()

  selected_dates = st.date_input(
      label="Rentang Waktu",
      min_value=min_date,
      max_value=max_date,
      value=[min_date, max_date]
  )

  # If only one date is selected, set start_date and end_date to the same date
  if len(selected_dates) == 1:
      start_date = selected_dates[0]
      end_date = selected_dates[0]
  else:
      start_date, end_date = selected_dates

# Define dataframe filter
main_df = p_polution_final_df[(p_polution_final_df["date"] >= start_date) & (p_polution_final_df["date"] <= end_date)]
main_df = main_df.drop(columns=["date"])



# """
# Segment 3 : Define the Variabel That will Use for Dashboard Visualization

# This section will add some visulization that will use in dashboard that also correlated
# with business question. The visualization that will use wil same as the section
# Data Visualization before.
# """

# Initialize Header Dashboard
st.header("Dicoding Air Quality Dashboard 🆘")

# Variabel for Visualization
st.subheader("Select Parameter and Type Visualization")
parameter = st.radio(
    "Choose Parameter Visualization:",
    ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3"],
    horizontal=True
)

visualization_type = st.radio(
    "Choose Visualization Type:",
    ["Annual", "Monthly", "Daily", "Hours"],
    horizontal=True
)

title= f"Trend of {parameter} - {visualization_type} Aggregation"
title_bar= f"Best and Worst of {parameter} - {visualization_type} Aggregation"

# Visualize 1 : Make Line Visualization for Trend Pollution Parameter
st.subheader("Trend Polution Parameter 📈")
st.markdown("This chart shows the trend of the selected air quality parameter over time.")

agg_df, id_vars, labels = aggregate_data(main_df, visualization_type)
fig = ploty_line_visualization(agg_df, id_vars, labels, [parameter], title_template=title)
st.plotly_chart(fig, use_container_width=True)

explanation = """
- Visualisasi dengan menggunakan line plot dapat melihat trend data baik dalam kategori Tahunan, Bulanan, Harian, dan bahkan Jam.
- Dalam menggunakannya pastikan sudah menetapkan rentangan tanggal dan memilih tipe visualisasi yang sesuai.
- Jika rentangan tahunan, maka visualisasi yang dapat ditampilkan adalah trend antar tahun, bulan, hari, dan jam.
- Sebaliknya jika rentangan hanya 1 bulan, maka visualisasi yang muncul nanti hanya rentangan hari, dan juga bisa memilih rentangan jam.
"""

with st.expander("Explanation: "):
    st.markdown(explanation)

# Visualize 2: Make Bar Visualization for Show Best and Worst Year, Month, Day, and Hours in Polution Parameter
st.subheader("Best and Worst Polutan Parameter According to Datetime 📊 ")
st.markdown("This bar chart shows the worst and best selected air quality parameter in a time.")

bar_fig = ploty_bar_visualization(agg_df, id_vars, labels, [parameter], title_template=title_bar)
st.plotly_chart(bar_fig, use_container_width=True)

explanation = """
- Penggunaan visualisasi bar plot ini menunjukkan perbandingan data baik dalam tahun, bulan, hari, dan jam.
- Jika tidak ada data yang dibandingkan maka visualisasi akan menggunakan data yang ada. Sebagai contoh:
  - Jika memilih rentang nilai dalam satu bulan, maka jika memilih tipe visualisasi 'Annual'
    visualisasi akan tetap jalan dengan data keseluruhan di tahun itu.
  - Ini sama dengan jika memilih tipe visualisasi 'Monthly' maka tetap menampilkan visualisasi hanya pada bulan itu saja.
  - Akan tetapi jika memlih tipe visualisasi 'Daily' atau 'Hourly' baru akan ada nilai yang dibandingkan.
- Nilai pada data sudah dilakukan proses sorting jadi data teratas adalah data dengan konsentrasi tertinggi
  baik pada tipe Tahun, Bulan, Hari, atau Jam.
"""

with st.expander("Explanation: "):
    st.markdown(explanation)

# Visualize 3: Make Geospasial Visualization for Area of Polution
st.subheader("Area of Polution 🌍")
st.markdown("This map shows the distribution of the selected air quality parameter across different locations.")

stations_coords = {
    "Aotizhongxin": (41.731242, 123.456778),
    "Changping": (40.221, 116.2312),
    "Dingling": (40.28998423518348, 116.2393424781757),
    "Dongsi": (40.10208908941478, 116.31657335910373),
    "Guanyuan": (39.94113871141321, 116.3610710753842),
    "Gucheng": (39.91270053243136, 116.1868698799306),
    "Huairou": (43.06043347888646, 117.46726428196578),
    "Nongzhanguan": (39.93978579546827, 116.46859787734736),
    "Shunyi": (40.151287025024715, 116.69280368021326),
    "Tiantan": (39.88189413732897, 116.42047003643812),
    "Wanliu": (39.99843210685499, 116.25774299569612),
    "Wanshouxigong": (39.90816416629832, 116.26439549963654)
}

spatial_df = final_df.copy()
spatial_df['date'] = pd.to_datetime(spatial_df["datetime"]).dt.date
spatial_df = spatial_df[(spatial_df["date"] >= start_date) & (spatial_df["date"] <= end_date)]
spatial_df.drop(columns=["date"])

spatial_df["latitude"] = spatial_df["station"].map(lambda x: stations_coords[x][0])
spatial_df["longitude"] = spatial_df["station"].map(lambda x: stations_coords[x][1])
spatial_df["datetime"] = pd.to_datetime(spatial_df["datetime"])

st.markdown("#### Select Geospasial Datetime")
col1, col2, col3, col4 = st.columns([1, 1, 1, 1])

all_cols = [col1, col2, col3, col4]
all_filters, filtered_df = filter_selected_datetime(spatial_df, all_cols)

fig = ploty_geospatial_visualization(filtered_df, all_filters, [parameter])
st.plotly_chart(fig, use_container_width=True)

explanation = """
- Ini adalah persebaran parameter polusi sesuai dengan latitude dan longitude-nya.
- Dalam persebaran ini diharuskan memilih persebaran sesuai dengan tahun, lalu bulan, hari, dan waktunya.
- Jika peta persebaran tidak langsung menunjukkan lokasinya maka bisa dikatakan dalam waktu itu tidak ada
data persebaran yang tersedia. Sebagai contoh:
  - Memilih tanggal 1 Januari 2023 tidak akan memunculkan nilai persebaran peta. Karena tidak ada data pada tanggal tersebut.
"""

with st.expander("Explanation: "):
    st.markdown(explanation)

# Visualize 4: Make Matrix Correlation for Parameter in Air Quality Index
st.subheader("Matrix Correlation Polutan Parameter 💐")
st.markdown("This matrix shows the correlation between different air quality parameters.")

check_correlation(final_df)

explanation = """
- Korelasi adalah ukuran statistik yang menunjukkan sejauh mana dua variabel beruhubungan.
- Nilai koefisien korelasi berkisar dari -1 hingga 1:
  - **1** menunjukkan korelasi positif sempurna (kedua variabel meningkat bersama).
  - **-1** menunjukkan korelasi negatif sempurna (saat satu variabel meningkat, variabel lainnya menurun).
  - **0** menunjukkan tidak ada korelasi antara variabel-variabel tersebut.
- Pada heatmap ini, warna yang lebih gelap mewakili korelasi yang lebih kuat, dengan warna biru menunjukkan korelasi negatif dan merah menunjukkan korelasi positif.
- Korelasi positif menunjukkan bahwa saat satu variabel meningkat, yang lainnya cenderung meningkat juga, dan sebaliknya untuk korelasi negatif.
"""

with st.expander("Explanation: "):
    st.markdown(explanation)


### **3. Tahap Menjalankan Dasboard**

#### 🏃🏻‍♀️ **Start Server**

In [ ]:
def run_streamlit_app(app_name, port=8501):
    """
    Runs a Streamlit app and sets up an ngrok tunnel to expose it to the internet.

    Parameters:
    ----------
    app_name : str
        The name of the Streamlit app file (e.g., "app.py").
    port : int, optional, default=8501
        The port on which to run the Streamlit app (default is 8501).

    Returns:
    --------
    str
        The public URL provided by ngrok for the Streamlit app.
    """
    # Kill any previous tunnels
    ngrok.kill()

    # Run the Streamlit app
    subprocess.Popen(["streamlit", "run", app_name, "--server.port", str(port)])

    # Start a new ngrok tunnel
    ngrok_tunnel = ngrok.connect(port, "http")

    # Print the public URL for the Streamlit app
    print("Streamlit URL:", ngrok_tunnel.public_url)

    return ngrok_tunnel.public_url

In [ ]:
from pyngrok import ngrok
import subprocess

app_name = "dashboard/dashboard.py"
run_streamlit_app(app_name)

#### ❌ **Shutdown Server**


In [ ]:
!ps aux | grep ngrok

In [ ]:
ngrok.kill()

In [ ]:
!ps aux | grep ngrok